In [2]:
import subprocess
import json

In [3]:
import os
from dotenv import load_dotenv
from constants import *
from bit import Key, PrivateKey, PrivateKeyTestnet
from bit.network import NetworkAPI
from bit import *
from web3 import Web3
from eth_account import Account

In [4]:
command = './derive -g --mnemonic="father left twin fine chicken okay girl vapor scissors involve mushroom tip" --cols=path,address,privkey,pubkey --format=json'

In [5]:
p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
output, err = p.communicate()
p_status = p.wait()


In [6]:
#keys = json.loads(output)

In [7]:
#print(keys)


In [8]:
# nodes running 
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1.8545"))

# Loading Mnemonic 
mnemonic = os.getenv('MNEMONIC', "father left twin fine chicken okay girl vapor scissors involve mushroom tip")
print(mnemonic)


father left twin fine chicken okay girl vapor scissors involve mushroom tip


In [9]:
coins = {"eth", "btc-test", "btc"}
numderive = 3

In [10]:
def derive_wallets(mnemonic, coin, numderive):
    command = f'php ./hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="{mnemonic}" --numderive="{numderive}" --coin="{coin}" --format=json' 
    
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
   
    keys = json.loads(output)
    return  keys

In [11]:
keys = {}
for coin in coins:
    keys[coin]= derive_wallets(os.getenv('mnemonic'), coin, numderive=3)

In [12]:
eth_PrivateKey = keys["eth"][0]['privkey']
btc_PrivateKey = keys['btc-test'][0]['privkey']


print(json.dumps(eth_PrivateKey, indent=1, sort_keys=True))
print(json.dumps(btc_PrivateKey, indent=1, sort_keys=True))


"0xd36d0f9ff2f6dc80208cd19e0b7d939f11bd1a63d10c38cdc1c58f4dbbc95edc"
"cUMT7ptJgxrwSm9s5zSUCKUhSNVq1UC9v31vgoSusyukAciNMdDt"


In [13]:
print(json.dumps(keys, indent=3, sort_keys=True))

{
   "btc": [
      {
         "address": "1Hk26uv3aSo2WmKNPbeUGjNWPFDBLU2waW",
         "index": 0,
         "path": "m/44'/0'/0'/0/0",
         "privkey": "KzQ3o74J1R3MFuzvRNQWjckRz3AYZHgggJRbs3P7TH1ZoXJqKety",
         "pubkey": "0399c9a8f6933934bfe811fc524c0e8684f971819398a0581f20ddf6371db0bb18",
         "pubkeyhash": "b7a4d19972d97725ebe1d980ac875f4b41322ec5",
         "xprv": "xprvA4EtnVRNTKMCodewrh6ipRZg96w7tyJMntxXmtPitFkMn7Dwe6BRfeC2bnL8DRs96stL1Kn752eZJqfFX1Fs25uaRxnUSSShU91qzeg5gWK",
         "xpub": "xpub6HEFBzxGHguW27jQxidjBZWQh8mcJS2DA7t8aGoLSbHLeuZ6BdVgDSWWT5kbpKZxGjjA8qDfgYCyKgBeau5Qp48c5KC9dTaYjWd3jLfF8MG"
      },
      {
         "address": "16LsmsW5nshaMWz4XXE9qZjE1HHEEF61Ma",
         "index": 1,
         "path": "m/44'/0'/0'/0/1",
         "privkey": "L3PRGxRUzyYGDEnWsdtyunpa5DCfYcgKfBSCCXGEHofUyxYj7zc6",
         "pubkey": "03412c487c7a81706519d3236215f717b9134a39e0447e1a70de08fd65d83ab7e1",
         "pubkeyhash": "3a9aea7796dbbce5a965d1fa4a7cf01cec335e7d",
    

In [14]:
def priv_key_to_account(coin, priv_key):
    """Convert the privkey string in a child key to an account object that bit or web3.py can use to transact"""
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)
    
eth_acc = priv_key_to_account(ETH,eth_PrivateKey)
btc_acc = priv_key_to_account(BTCTEST,btc_PrivateKey)

In [15]:
print(eth_acc)
print(btc_acc)

<PrivateKeyTestnet: mkwqRYLAfeoMJnnb5A9WegujdggyBeEPGD>


In [52]:
def create_trx(coin, account, recipient, amount):
    """create the raw, unsigned transaction that contains all metadata needed to transact"""
    global trx_data
    if coin ==ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        trx_data = {
            "chainId" : 100,
            "to": recipient,
            "from": account.address,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address)
        }
        return trx_data

    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])

In [53]:
def send_trx(coin, account, recipient, amount):
    """call create_trx, sign the transaction, then send it to the designated network"""
    if coin == "eth": 
        trx_eth = create_trx(coin,account, recipient, amount)
        sign = account.signTransaction(trx_eth)
        result = w3.eth.sendRawTransaction(sign.rawTransaction)
        print(result.hex())
        return result.hex()
    else:
        trx_btctest= create_trx(coin,account,recipient,amount)
        sign_trx_btctest = account.sign_transaction(trx_btctest)
        from bit.network import NetworkAPI
        NetworkAPI.broadcast_tx_testnet(sign_trx_btctest)       
        return sign_trx_btctest

### Create transaction for BTC 

In [17]:
create_trx(BTCTEST,btc_acc,"mtXBtkFWdG34JVmKzfVMUT2vYa5Q8DoRjh", 0.001)

'{"unspents":[{"amount":8174985,"confirmations":47,"script":"76a9143b8d4adb98c1922686a0b9b3f10ee0f1ef7a2eb888ac","txid":"ae4588acf593d4c812b0aff0bc422f6be0678c20b15f72c4c3c4d4d99ab653ce","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["mtXBtkFWdG34JVmKzfVMUT2vYa5Q8DoRjh",100000],["mkwqRYLAfeoMJnnb5A9WegujdggyBeEPGD",8051933]]}'

In [18]:
send_trx(BTCTEST,btc_acc,"mtXBtkFWdG34JVmKzfVMUT2vYa5Q8DoRjh",0.001)

'0100000001ce53b69ad9d4c4c3c4725fb1208c67e06b2f42bcf0afb012c8d493f5ac8845ae010000006b483045022100f3b6a986d8253d3b705a8520225a00043dc28023a0f68d9e7d93da8508b3ef5202206622776bef1f1e676459d69d37cbcbafd47982ea2243a73a4af1ff9fdd10b43b012102dcb7a11c7172966e58679c0cb207bfb95d119cc55f9fb318d0a522aaff5b9577ffffffff02a0860100000000001976a9148ea4fc816fbebe9d5f6509f9367240e6b355af9b88acdddc7a00000000001976a9143b8d4adb98c1922686a0b9b3f10ee0f1ef7a2eb888ac00000000'

### Local PoA Ethereum transaction

In [29]:
from web3.middleware import geth_poa_middleware
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [30]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3 

In [31]:
load_dotenv()

True

In [32]:
private_key = os.getenv("PRIVATE_KEY")

In [33]:
account_one = Account.from_key(private_key)

In [34]:
private_key2 = os.getenv("PRIVATE_KEY2")
account_two = Account.from_key(private_key2)

In [35]:
account_one.address

'0x943becD089023d89531c56e8c31bBD670A80CC1D'

In [36]:
account_two.address

'0x21ff02C61E69584d4aAe6b3aE80374D01778084f'

In [37]:
private_key2 = os.getenv("PRIVATE_KEY1")
account_node1 = Account.from_key(private_key2)
account_node1.address

'0x66AdBb47BE34D5143DBBe93ecf9af76352feC491'

In [38]:
w3.eth.getBalance("0x943becD089023d89531c56e8c31bBD670A80CC1D")

904625697166532776746648320380374280103671755200316906358261115061821325312

### ETH Transacation 

In [41]:
# using the create_trx function 

create_trx(ETH,eth_acc,'0x943becD089023d89531c56e8c31bBD670A80CC1D', 1)

{'to': '0x943becD089023d89531c56e8c31bBD670A80CC1D',
 'from': '0x02194a55DDA1B029F70220D53f099F38c629ab36',
 'value': 1,
 'gasPrice': 1000000000,
 'gas': 21000,
 'nonce': 0}

In [54]:
# using the send_trx function 

send_trx(ETH, eth_acc,"0x943becD089023d89531c56e8c31bBD670A80CC1D", 1)

0xad780e0ce2ac1ba8c404522423ca77f92c65f74a11b544549167a9097c7b8207


'0xad780e0ce2ac1ba8c404522423ca77f92c65f74a11b544549167a9097c7b8207'

### Check balance 

In [55]:
w3.eth.getBalance(account_one.address)

904625697166532776746648320380374280103671755200316906258260695061821325313